# Setup

In [1]:
%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

True

# RAG Search Example

In [8]:
%pip install langchain==0.1.7 docarray tiktoken

   ---------------------------------------- 0.0/815.9 kB ? eta -:--:--
    --------------------------------------- 10.2/815.9 kB ? eta -:--:--
   ---- ----------------------------------- 81.9/815.9 kB 1.1 MB/s eta 0:00:01
   ---------------- ----------------------- 337.9/815.9 kB 3.0 MB/s eta 0:00:01
   ---------------------------------------  809.0/815.9 kB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 815.9/815.9 kB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ----------------- ---------------------- 0.7/1.7 MB 22.8 MB/s eta 0:00:01
   ------------------------------------- -- 1.6/1.7 MB 19.8 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.19
    Uninstalling langchain-community-0.0.19:
      Successfully uninstalled langchain-community-0.0.19
  Attempting uninstall: langcha

In [12]:
# workaround for an error with pydantic 2
%pip install pydantic==1.10.14

     ---------------------------------------- 0.0/151.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/151.6 kB ? eta -:--:--
     -- ------------------------------------- 10.2/151.6 kB ? eta -:--:--
     ------- ----------------------------- 30.7/151.6 kB 330.3 kB/s eta 0:00:01
     ------------------- ----------------- 81.9/151.6 kB 573.4 kB/s eta 0:00:01
     ------------------------------------ 151.6/151.6 kB 905.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ------- -------------------------------- 0.4/2.1 MB 12.6 MB/s eta 0:00:01
   -------------- ------------------------- 0.8/2.1 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 14.9 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 13.5 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.1
    Uninstalling pydantic-2.6.1:
      Successfully uninstalled pydantic-2.6.1
Note

In [2]:
# Requires:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [3]:
retriever.invoke("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [5]:
retriever.invoke('')

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [7]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
setup_and_retrieval

{
  context: VectorStoreRetriever(tags=['DocArrayInMemorySearch'], vectorstore=<langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x00000230F38071D0>),
  question: RunnablePassthrough()
}

In [11]:
setup_and_retrieval.invoke('What is love?')

{'context': [Document(page_content='bears like to eat honey'),
  Document(page_content='harrison worked at kensho')],
 'question': 'What is love?'}

In [12]:
(setup_and_retrieval | prompt).invoke('What is love?')

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\n[Document(page_content='bears like to eat honey'), Document(page_content='harrison worked at kensho')]\n\nQuestion: What is love?\n")])